In [1]:
from keras.layers import Input, Conv1D, Flatten, Dropout, MaxPooling1D, Dense, BatchNormalization, Activation, UpSampling1D, Concatenate
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from keras.models import Model
from keras import optimizers
from os.path import join
from time import time
import numpy as np
import random
import h5py
import glob
import os

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
hf = h5py.File('wavelet_data_25.h5', 'r')
X = np.array(hf.get('data'))
y = np.array(hf.get('labels'))
hf.close()

In [3]:
labels = np.where( y == 7 )
y = np.delete(y, labels, axis=0)
X = np.delete(X, labels, axis=0)
y[y > 7] -= 1

In [4]:
X = np.reshape(X, (X.shape[0], np.prod(X.shape[1:])))
y = np.eye(24)[y]
print(X.shape, y.shape)

(959, 4320000) (959, 24)


In [5]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

history = []
cvscores = []

for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    scaler = StandardScaler()
    scaler.fit(X_train)

    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)    
    
    pca = PCA(n_components=0.95)
    pca.fit(X_train)

    X_train = pca.transform(X_train)
    X_test = pca.transform(X_test)    
    
    X_train = np.expand_dims(X_train, axis = -1)
    X_test = np.expand_dims(X_test, axis = -1)

    inputs = Input(shape=(X_train.shape[-2], 1))

    x = Conv1D(16, 17, strides=1, padding='same')(inputs)
    x = BatchNormalization(axis = -1)(x)
    x = Activation('selu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(16, 11, strides=1, padding='same')(x)
    x = BatchNormalization(axis = -1)(x)
    x = Activation('selu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(16, 11, strides=1, padding='same')(x)
    x = BatchNormalization(axis = -1)(x)
    x = Activation('selu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(16, 7, strides=1, padding='same')(x)
    x = BatchNormalization(axis = -1)(x)
    x = Activation('selu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(16, 7, strides=1, padding='same')(x)
    x = BatchNormalization(axis = -1)(x)
    x = Activation('selu')(x)
    x = MaxPooling1D(2)(x)

    x = Flatten()(x)
    x = Dense(24, activation='linear')(x)
    x = Activation('softmax')(x)

    model = Model(inputs=inputs, outputs=x)

    model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=1e-2, decay=1e-5), metrics=['acc'])

    history.append(model.fit(x=X_train, y=y_train, epochs=20, validation_data=(X_test, y_test), verbose=0))
    
    cvscores.append(history[-1].history['val_acc'][-1] * 100)
    
    print("mean acc: %.2f%% (+/- %.2f%%), min: %.2f%%, max: %.2f%%, count: %i" % (np.mean(cvscores), np.std(cvscores), np.min(cvscores), np.max(cvscores), len(cvscores)))

mean acc: 86.98% (+/- 0.00%), min: 86.98%, max: 86.98%, count: 1.00%


In [19]:
hf = h5py.File('wavelet_data_corrupt.h5', 'r')
X_c = np.array(hf.get('data'))
y_c = np.array(hf.get('labels'))
hf.close()

X_c = np.reshape(X_c, (X_c.shape[0], np.prod(X_c.shape[1:])))
X_c = scaler.transform(X_c)
X_c = pca.transform(X_c)
X_c = np.expand_dims(X_c, axis = -1)

In [22]:
hf = h5py.File('environment_data.h5', 'r')
X_e = np.array(hf.get('data'))
y_e = np.array(hf.get('labels'))
hf.close()

X_e = np.reshape(X_e, (X_e.shape[0], np.prod(X_e.shape[1:])))
X_e = scaler.transform(X_e)
X_e = pca.transform(X_e)
X_e = np.expand_dims(X_e, axis = -1)

In [23]:
print(np.sort(np.amax(model.predict(X_e), axis=1)))

[0.21854565 0.2536702  0.2538617  0.26514244 0.27193734 0.2749421
 0.28777477 0.3306155  0.35680595 0.38206664 0.41067147 0.42178515
 0.4327927  0.4440774  0.45466504 0.51565224 0.52653223 0.5322107
 0.54501724 0.54949194 0.5780926  0.6230369  0.6242679  0.6599329
 0.67505664 0.72023463 0.7218896  0.72503185 0.78266    0.82011825
 0.84132403 0.90320057 0.90492094 0.9380708  0.94952357 0.96109396
 0.96908253 0.9728919  0.9731391  0.9743329  0.9790932  0.9791798
 0.983085   0.98317105 0.98323613 0.985748   0.98698795 0.98710144
 0.98908323 0.98960614 0.9935823  0.99494237 0.99723047 0.99736756
 0.9981343  0.9988292  0.99891996 0.9990213  0.99939704 0.9995746
 0.99979097 0.9998467  0.9998568  0.99986994 0.99987674 0.999925
 0.99993145 0.9999447  0.9999896 ]


In [20]:
print(np.sort(np.amax(model.predict(X_c), axis=1)))

[0.29377568 0.3905207  0.44930288 0.4601511  0.47530574 0.5035693
 0.5300932  0.5763319  0.5925576  0.6310839  0.6381617  0.7152465
 0.73163825 0.73616767 0.76218015 0.78554475 0.78969485 0.8522823
 0.86340714 0.8758991  0.88999444 0.8957597  0.89879495 0.90005076
 0.9025091  0.93440187 0.945672   0.9488211  0.9557062  0.9605283
 0.96210766 0.97465056 0.9751152  0.9751721  0.993053   0.99598736
 0.99624133 0.9965957  0.9972364  0.9985459  0.9989453  0.99909246
 0.9990946  0.9992217  0.9992293  0.99923503 0.9993254  0.9993887
 0.99943143 0.9994423  0.9994524  0.9994771  0.9994856  0.9994906
 0.9995246  0.9995377  0.9995554  0.9995586  0.9995994  0.99962497
 0.99963295 0.9996439  0.99966764 0.99966884 0.99968874 0.99969006
 0.99971396 0.9997191  0.99974865 0.99976486 0.99976844 0.9997813
 0.9997868  0.99978787 0.99978787 0.99979013 0.9997913  0.99979585
 0.99979776 0.99982375 0.9998344  0.9998485  0.99985373 0.99985373
 0.99985456 0.9998568  0.9998586  0.9998692  0.99987185 0.9998821
 0.

In [16]:
print(np.sort(np.amax(model.predict(X_test), axis=1)))

[0.26010048 0.29181194 0.34775043 0.36208484 0.39577612 0.46337318
 0.48196647 0.49883193 0.5000483  0.5139324  0.5408565  0.56609625
 0.5879594  0.6114858  0.6129101  0.6452812  0.67696685 0.68123335
 0.68131137 0.69983155 0.70597273 0.73281974 0.7350025  0.74915683
 0.7595967  0.7653355  0.77652603 0.77960736 0.7848361  0.7919956
 0.7928235  0.81166506 0.8116808  0.82016385 0.84569395 0.847277
 0.85542244 0.8631906  0.8723153  0.8755788  0.8759258  0.87642473
 0.87685704 0.8821573  0.8875847  0.8945708  0.8997404  0.90979475
 0.91847515 0.92164713 0.92557824 0.9264355  0.9310173  0.93190604
 0.9334066  0.93528074 0.9363084  0.9365047  0.9428726  0.9440619
 0.9441     0.94559056 0.9575736  0.9590398  0.9619312  0.9674941
 0.9691408  0.97192556 0.97195    0.976824   0.97852546 0.9789763
 0.98022217 0.9808456  0.98132735 0.98133576 0.98428464 0.9844327
 0.98517376 0.98552763 0.9855392  0.9857174  0.9892938  0.98930055
 0.99034494 0.9905381  0.9920034  0.9921523  0.9924555  0.9927563
 0.

In [ ]:
val_acc = []
acc = []

val_loss = []
loss = []

for i in history:
    val_acc.append(i.history["val_acc"])
    acc.append(i.history["acc"])
    val_loss.append(i.history["val_loss"])
    loss.append(i.history["loss"])
    
val_acc = np.array(val_acc)
acc = np.array(acc)
val_loss = np.array(val_loss)
loss = np.array(loss)

np.save("acc", acc)
np.save("val_acc", val_acc)
np.save("loss", loss)
np.save("val_loss", val_loss)